You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

## 1. Set up

### 1 - Drive

1 - Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


2 - Move to the data folder

In [2]:
cd "gdrive/MyDrive/Projects/1 - Numericals/Predict Future Sales/2 - Production/data"

/content/gdrive/MyDrive/Projects/1 - Numericals/Predict Future Sales/2 - Production/data


### 2. Libraries

In [3]:
# Load data
import pandas as pd
import numpy as np
import io
import os
import glob

# Meta
import time

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sb

# Analysis
from scipy.stats import zscore

## 3. Data

1 - List file names

In [4]:
ls

competitive-data-science-predict-future-sales.zip  sales_train.csv
item_categories.csv                                sample_submission.csv
items.csv                                          shops.csv
kaggle.json                                        test.csv


2 - Load files

In [5]:
df_item_categories = pd.read_csv('item_categories.csv')
df_items = pd.read_csv('items.csv')
df_sales_train = pd.read_csv('sales_train.csv')
df_sample_submission = pd.read_csv('sample_submission.csv')
df_shops = pd.read_csv('shops.csv')
df_test = pd.read_csv('test.csv')

### 4. Classes

1 - Define the class

A - Data Assessment of one class

In [6]:
# Class that helps the assessment of each table individually
class DataAssessment:

  def __init__(self, df):
    self.df = df


  # Get the basic file information
  def files_basic_info(self):
    '''
    Function - Get the very basic details about the data files
    Input - None
    Action - Find the number of csv files, file size,
            number of rows and number of columns
    Dependencies - 
      import glob
      import os
    '''
    begin = time.time()
    
    # Get all the files in the folder
    files = os.listdir()
    file_list = glob.glob('*.csv')

    # Find the number of files in the folder
    print('The number of files in the directory is:',len([name for name in os.listdir('.') if os.path.isfile(name)]))
    print('\n')
    print('The file names are:')
    print(files)
    print('\n')

    # Get the details of the csv files
    print('The csv files details are:')
    print('\n')
    for i in file_list:
      print('File:',i)
      file_size = os.path.getsize(i)
      converted_size = self.formatFileSize(file_size, 'B', 'MB', precision=0)
      df = pd.read_csv(i,error_bad_lines=False)
      df_shape = df.shape
      n_rows = df_shape[0]
      n_columns = df_shape[1]
      file_size = self.formatFileSize(file_size, 'B', 'MB', precision=2)
      head = df.head()
      print('File size:',file_size,'MB')
      print('Number of data points:',n_rows)
      print('Number of features:',n_columns)
      print('\n')
    end = time.time()
    self.find_time_taken(begin, end)

  
  # Code attribution : https://www.codegrepper.com/code-examples/python/convert+bytes+to+mb+python
  def convertFloatToDecimal(self, f=0.0, precision=2):
      '''
      Function: Convert a float to string of decimal.
      precision: by default 2.
      If no arg provided, return "0.00".
      '''
      return ("%." + str(precision) + "f") % f


  # Code attribution : https://www.codegrepper.com/code-examples/python/convert+bytes+to+mb+python
  def formatFileSize(self, size, sizeIn, sizeOut, precision=0):
    '''
    Function: Convert file size to a string representing its value in B, KB, MB and GB.
    The convention is based on sizeIn as original unit and sizeOut
    as final unit. 
    '''
    assert sizeIn.upper() in {"B", "KB", "MB", "GB"}, "sizeIn type error"
    assert sizeOut.upper() in {"B", "KB", "MB", "GB"}, "sizeOut type error"
    if sizeIn == "B":
        if sizeOut == "KB":
            return self.convertFloatToDecimal((size/1024.0), precision)
        elif sizeOut == "MB":
            return self.convertFloatToDecimal((size/1024.0**2), precision)
        elif sizeOut == "GB":
            return self.convertFloatToDecimal((size/1024.0**3), precision)
    elif sizeIn == "KB":
        if sizeOut == "B":
            return self.convertFloatToDecimal((size*1024.0), precision)
        elif sizeOut == "MB":
            return self.convertFloatToDecimal((size/1024.0), precision)
        elif sizeOut == "GB":
            return self.convertFloatToDecimal((size/1024.0**2), precision)
    elif sizeIn == "MB":
        if sizeOut == "B":
            return self.convertFloatToDecimal((size*1024.0**2), precision)
        elif sizeOut == "KB":
            return self.convertFloatToDecimal((size*1024.0), precision)
        elif sizeOut == "GB":
            return self.convertFloatToDecimal((size/1024.0), precision)
    elif sizeIn == "GB":
        if sizeOut == "B":
            return self.convertFloatToDecimal((size*1024.0**3), precision)
        elif sizeOut == "KB":
            return self.convertFloatToDecimal((size*1024.0**2), precision)
        elif sizeOut == "MB":
            return self.convertFloatToDecimal((size*1024.0), precision)


    # Show the first 10 rows of the dataframe
  def print_head(self, df):
    '''
    Function: Print the first 10 rows of the dataframe 
    Input: A dataframe
    Output: The first 10 rows of the dataframe
    '''
    return df.head(n=10)
  
  # Show a single datapoint vertically
  def print_vertically(self, df):
    '''
    Function: Print the first data point vertically
    Input: A dataframe
    Output: The first data point vertically
    '''
    return df.iloc[0]

  # Find duplicates
  def find_duplicates(self, df):
    '''
    Function - Find and display duplicate columns and rows of the dataframe
    Input - Dataframe
    Output - Display duplicate columns and rows of the dataframe
    '''
    begin = time.time()

    # Find duplicate rows
    row_duplicates = df[df.duplicated()]
    
    # Find duplicate columns
    duplicate_columns = set()
    other_columns = set()
    
    for x in range(df.shape[1]):
      col = df.iloc[:, x]
      for y in range(x + 1, df.shape[1]):
        otherCol = df.iloc[:, y]
        
        if col.equals(otherCol):
          duplicate_columns.add(df.columns.values[y])
          other_columns.add(df.columns.values[x])
    
    dup_columns = (list(duplicate_columns))
    dup_other_columns = (list(other_columns))

    
    columns = 2
    rows = len(dup_columns)
    column_duplicates = []
    column_duplicates = [[0 for i in range(columns)] for j in range(rows)]
    
    for i in range(len(column_duplicates)):
      val_one = dup_columns[i]
      val_two = dup_other_columns[i]
      column_duplicates[0][i] = val_one
      column_duplicates[i][1] = val_two
    print('\nRow duplicates:')
    print(row_duplicates)
    print('\n')
    print('\nColumn duplicates:')
    print(column_duplicates)

    end = time.time()
    time_taken = end - begin


  # Check the data types of the dataframe
  def check_feature_types(self, df):
    '''
    Function: Get basic information on the dataframe 
    Input: Dataframe
    Output: Some basic information on the dataframe 
    '''
    begin = time.time()

    # Get the information from the dataframe

    # Get the shape of the dataframe
    data_shape = df.shape
    # Get the data types
    data_types = df.dtypes
    # Get the number of rows and columns
    n_rows = data_shape[0]
    n_columns = data_shape[1]
    # Find the missing values
    null_values = df.isnull().sum()
    # Find numeric columns and categorical columns
    numeric_columns = df.select_dtypes([np.number]).columns.tolist()
    columns = list(df)
    categorical_columns = []
    for i in columns:
      if i not in numeric_columns:
        categorical_columns.append(i)
      else:
        None
    
    print('\nCategorical columns:',len(categorical_columns))
    print(categorical_columns)
    print('\nNumeric columns:',len(numeric_columns))
    print(numeric_columns)
    print('\n')
    print('The column type information:')
    df.info()
    end = time.time()
    
    # Display the time taken to run this procedure
    end = time.time()
    self.find_time_taken(begin, end)


  # Function to calculate the time taken to run the function that calls it
  def find_time_taken(self, begin, end):
    '''
    Function: Calculate the time taken to run this procedure
    Input: Beginning time, ending time
    Action: Calculate the time taken for a procedure to be run in minutes or seconds
    '''
    time_taken = end - begin

    if time_taken >= 60:
      time_taken = round(time_taken/60,2)
      print("Time taken to run this procedure:",time_taken,"minutes")
    else:
      time_taken = round(time_taken,2)
    print('\n')
    print("Time taken to run this procedure:",time_taken,"seconds")

  
  # Missing values analysis
  def missing_data_basic_analysis(self, df):
    '''
    Function: Provide analysis on the missing values in the dataframe
    Input: Dataframe
    Output: Row-wise and column-wise analysis of NaN values in the dataframe
    '''

    begin = time.time()

    data_shape = df.shape
    n_rows = data_shape[0]
    n_columns = data_shape[1]
    null_values = df.isnull().sum()
    columns_nans = df.isnull().sum()[df.isnull().sum() > 0]
    n_columns_nans = len(columns_nans)
    percent_nan_columns = round((columns_nans/n_rows)*100,2).sort_values()
    percent = percent_nan_columns
    drop_columns = percent.where(percent > 30)
    drop_columns = drop_columns.dropna()
    n_drop_columns = len(drop_columns)

    all_nan_rows = df[(df.T.isnull()).all()]

    nan_count_column = []
    for index, row in df.iterrows():
      nan_count = df.loc[[index]].isna().sum().sum()
      nan_count_column.append(round((nan_count/n_columns)*100,2))
    df['NaN %'] = nan_count_column
    df.sort_values(by=['NaN %'])
    
    print('Column Analysis:')
    print('\nColumns with null values:')
    print('There are',n_columns_nans,'columns with null values which is',round((n_columns_nans/n_columns)*100,0),'percent of the features')
    print('\nColumns with NaNs:')
    print(percent_nan_columns)
    print('\nColumns with NaNs greater than 30%:')
    print(drop_columns)
    print('\nNumber of columns to be dropped:')
    print(n_drop_columns,'needs to be dropped, that is,',round((n_drop_columns/n_columns)*100,0),'% of total features')
    print('\nDescriptive analysis of columns with missing values')
    print(percent_nan_columns.describe())

    print('\nRow Analysis:')
    print('\nThe number of rows with all NaN values are:',all_nan_rows.shape[0])
    df.drop('NaN %', inplace=True, axis=1)
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
    

  # Analyze the zero values in the dataframe
  def zero_value_analysis(self, df):
    '''
    Function: Get row-wise and column-wise analysis of 0 values in the dataframe
    Input: Dataframe
    Output: Row-wise and column-wise analysis of 0 values in the dataframe
    '''

    begin = time.time()

    nulls = df.eq(0).sum() 
    column_1 = df.columns
    
    column_2 = nulls.tolist()
    
    dataframe_shape = df.shape
    dataframe_rows = dataframe_shape[0]
    dataframe_columns = dataframe_shape[1]
    column_3 = []
    for i in column_2:
      column_3.append((round(i/dataframe_rows,2))*100)

    # Get a table with the column name, number of null values and percentage of null values
    final_data = {'Column Name':column_1, 'Zero Values':column_2, '% of Zeroes':column_3}
    null_values_table = pd.DataFrame(final_data,columns = ['Column Name','Zero Values','% of Zeroes'])
    null_values_table.sort_values(by=['% of Zeroes'], inplace=True, ascending=False)
    #selection = df.loc[df['mylist']==0]
    null_values_table = null_values_table.loc[null_values_table['Zero Values'] > 0]

    print("Columns:")
    print('\n')
    print('Number of columns with 0 values:',null_values_table.shape[0])
    print(null_values_table)

    print('\n')
    print('Rows:')
    print('\n')
    
    zero_rows = df[(df.T == 0).all()]
    n_zero_rows = zero_rows.shape[0]
    print('The number of rows with only zero values is:',n_zero_rows)
    print('The percentage of rows with only zero values is:',round((n_zero_rows/dataframe_rows)*100,2))
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
  

  # Cardinality of features
  def display_cardinality(self, df):
    '''
    Function: Show cardinality of features in the datframe
    Input: A dataframe
    Output: Column names and their cardinality
    '''
    begin = time.time()
    unique_values = df.nunique()
    print(unique_values)
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
  

  # Show all unique values in a single column
  def unique_values(self, df, column_name):
    '''
    Function: Show unique values of the column
    Input: The dataframe and column name in string format
    Output: Unique values of a column
    '''

    begin = time.time()
    unique_values = df[column_name].unique()
    print(unique_values)
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)


  # Show the categorical and numeric features
  def feature_types(self, df):
    '''
    Function: Find the number and names of categorical and numeric features
    Input: Dataframe
    Output: The number and names of categorical and numeric features
    '''
    begin = time.time()

    # Get the information from the dataframe
    numeric_columns = df.select_dtypes([np.number]).columns.tolist()
    columns = list(df)
    categorical_columns = []
    for i in columns:
      if i not in numeric_columns:
        categorical_columns.append(i)
      else:
        None
    
    # Print the information
    print('\nCategorical columns:',len(categorical_columns))
    print(categorical_columns)
    print('\nNumeric columns:',len(numeric_columns))
    print(numeric_columns)
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
  

  # Check spaces in column headers
  def check_spaces(self, df):
    '''
    Functions: Check if column headers have spaces and return names of those that do
    Input: Dataframe
    Output: Names of column headers with spaces
    '''
    begin = time.time()

    list_1 = df.columns
    list_2 = [c.replace(' ', '_') for c in list_1]
    col_names = set(list_1)-set(list_2)
    return list(col_names)
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)


  # Numeric feature descriptive statistics
  def numeric_feature_details(self, df):
    '''
    Function: Display descriptive statistics of all numeric features in the dataframe
    Input: Dataframe
    Output: Descriptive statistics of all numeric features in the dataframe
    '''
    begin = time.time()

    # Get the information from the dataframe

    numeric_columns = df.select_dtypes([np.number]).columns.tolist()
    print('Number of numeric columns:', len(numeric_columns))
    print(numeric_columns)
    print('\n')

    for i in numeric_columns:
      print('Column Name:',i)
      print(df[i].describe())
      print('\n')
      print('median:',df[i].median())
      print('mode:',df[i].mode())
      print('\n')
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)


  def numeric_details_list(self, df, column_list):
    '''
    Function: Find the number and names of categorical and numeric features
    Input: Dataframe
    Output: The number and names of categorical and numeric features
    '''
    begin = time.time()

    # Get the information from the dataframe

    numeric_columns = column_list
    print('Number of numeric columns:', len(numeric_columns))
    print(numeric_columns)
    print('\n')

    for i in numeric_columns:
      print('Column Name:',i)
      print(df[i].describe())
      print('\n')
      print('median:',df[i].median())
      print('mode:',df[i].mode())
      print('\n')
    end = time.time()
    
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)

    #Time taken to run this procedure
    self.find_time_taken(begin, end)

  
  # Certain columns can have a standardized length - check values of such columns
  def check_value_length(self, df, column_name):
    '''
    Function: To find the length of a value in a given column
    Input: The dataframe and name of the column as a variable
    Output: The length of a value in a given column
    '''
    begin = time.time()
    df_copy = df.copy(deep=False)
    df_copy['character_length'] = df_copy[column_name].astype(str).map(len)
    print(df_copy['character_length'].unique())
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
  


  def get_column_lists(self, df):
    '''
    Function: Get the categorical and numerical column names of a dataframe as two lists
    Input: A dataframe
    Output: The categorical and numerical column name lists
    '''
    begin = time.time()

    numeric_columns = df.select_dtypes([np.number]).columns.tolist()
    columns = list(df)
    categorical_columns = []
    for i in columns:
      if i not in numeric_columns:
        categorical_columns.append(i)
      else:
        None
    end = time.time()
    
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
    return numeric_columns, categorical_columns
  

  def convert_columns_string(self, df):
    '''
    Function: Convert all columns in the given dataframe to string type
    Input: A dataframe
    Output: The dataframe with all columns in string type
    '''
    begin = time.time()
    df_copy = df.copy(deep=False)
    for i in df_copy:
        df_copy[i] = df_copy[i].astype(str)
    end = time.time()
  
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
    return df_copy
  

  def check_column_values(self, df, column_list):
    '''
    Function: Check for data quality in each value of each categorical column
    Input: The dataframe and the list of categorical columns
    Output: Presence of spaces, numeric values, decimals, all alphabets, digits, 
            if the value is in lower, upper and title cases
    Dependency: Use convert_columns_string() to convert all columns to string format. 
                Or use get_column_lists() to get numeric and categorical column name lists
    '''
    begin = time.time()
    try:
            print('\n')
            print('Check if all characters in the string are whitespaces. True if space exists, else False.')
            for i in column_list:
              print('Column name:',i)
              check = df[i].str.isspace()
              print(check.unique())

            print('\n')
            print('Check whether all characters are numeric. True if only numeric, else False.')
            for i in column_list:
              print('Column name:',i)
              check = df[i].str.isnumeric()
              print(check.unique())

            print('\n')
            print('Check whether all characters are only alphabetic. True if there are only alphabets, else False.')
            for i in column_list:
              print('Column name:',i)
              check = df[i].str.isalpha()
              print(check.unique())

            print('\n')
            print('Check whether all characters are only digits. True if there are only digits, else False.')
            for i in column_list:
              print('Column name:',i)
              check = df[i].str.isdigit()
              print(check.unique())

            print('\n')
            print('Check whether all characters are only decimal. True if there are only decimal, else False.')
            for i in column_list:
              print('Column name:',i)
              check = df[i].str.isdecimal()
              print(check.unique())

            print('\n')
            print('Check whether all characters are only lower. True if there are only lower case, else False.')
            for i in column_list:
              print('Column name:',i)
              check = df[i].str.islower()
              print(check.unique())

            print('\n')
            print('Check whether all characters are only upper. True if there are only upper case, else False.')
            for i in column_list:
              print('Column name:',i)
              check = df[i].str.isupper()
              print(check.unique())
            
            print('\n')
            print('Check whether all characters are only upper. True if there are only upper case, else False.')
            for i in column_list:
              print('Column name:',i)
              check = df[i].str.istitle()
              print(check.unique())
          
    except:
      print('\nPlease check the above column type. Only columns of string type are allowed.')
      print('Check if the column is numeric or boolean and remove them from the column_list list.')
    
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)


  def remove_elements_lists(self, main_list, remove_list):
    '''
    Function: Remove a set of elements from a list using another list
    Input: Two lists
    Output: The list where elements in the second lists are removed from the first list
    '''
    begin = time.time()
    final_list = []
    [final_list.append(x) for x in main_list if x not in remove_list]
    end = time.time()
      
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
    return final_list
  

  def replace_nans_zero(self, df, column_list):
    '''
    Function: Replace NaNs in the dataframe columns with 0
    Input: Dataframe, list of columns
    Output: Dataframe with values in specified column list 0
    '''
    begin = time.time()
    df_copy = df.copy(deep=False)
    for i in column_list:
      df_copy[i] = df_copy[i].replace(np.nan, 0)
    end = time.time()
      
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
    
    return df_copy
  

  def scatter_column_values(self, df, x_value, y_value):
    '''
    Function: Scatter plot between two columns in the same dataframe
    Input: Dataframe, x-value, y-value
    Output: Scatter plot
    Dependency: import matplotlib.pyplot as plt
    '''
    begin = time.time()
    df.plot(x=x_value, y=[y_value], kind='scatter')
    print(x_value,'vs.',y_value)
    print('\n')
    plt.figure(figsize=(3, 4))
    plt.show()

    end = time.time()
      
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
  

  def print_outlier_values(self, df, column_name):
    '''
    Function: Find the higher and lower outlier values of a column.
              Get two dataframes with only higher and lower values rows
    Input: Dataframe and a column name
    Outout: Two dataframes with higher and lower outliers respectively
    '''
    begin = time.time()
    quartiles = df[column_name].quantile([0.25,0.5,0.75])
    q1 = quartiles[0.25]
    q3 = quartiles[0.75]
    iqr = q3 - q1
    lower_outlier_filter = q1 - 1.5 * iqr
    higher_outlier_filter = q3 + 1.5 * iqr
    print('Higher outliers:',higher_outlier_filter)
    print('Lower outliers:',lower_outlier_filter)
    end = time.time()
      
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)


  def get_outlier_dfs(self, df, column_name):
    '''
    Function: Find the higher and lower outlier values of a column.
              Get two dataframes with only higher and lower values rows
    Input: Dataframe and a column name
    Outout: Two dataframes with higher and lower outliers respectively
    '''
    begin = time.time()
    quartiles = df[column_name].quantile([0.25,0.5,0.75])
    q1 = quartiles[0.25]
    q3 = quartiles[0.75]
    iqr = q3 - q1
    lower_outlier_filter = q1 - 1.5 * iqr
    higher_outlier_filter = q3 + 1.5 * iqr
    df_higher_outliers = df.loc[(df[column_name] > higher_outlier_filter)]
    df_lower_outliers = df.loc[(df[column_name] < lower_outlier_filter)]
    end = time.time()
      
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
    return df_higher_outliers, df_lower_outliers


  def get_outlier_normal_dist(self, df, column_name):
    '''
    Function: Get higher outlier dataframe based on the given column that is distributed normally
    Input: Dataframe, column name
    Output: Higher outlier dataframe
    Dependency: from scipy.stats import zscore
    '''
    begin = time.time()
    df_copy = df.copy(deep=False)
    new_col = column_name + '_zscore'
    df_copy[new_col] = zscore(df[column_name])
    df_higher_outliers = df_copy.loc[(abs(df_copy[new_col]) > 3)]
    end = time.time()
      
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
    return df_higher_outliers

  def get_index_list(self, df, series):
    '''
    Function: Get indices of the dataframe where the series value is True 
    Input: The dataframe, a series
    Output: The dataframe with indeices
    '''
    index_list = series.loc[series == True].index.tolist()
    df_index = df.iloc[index_list]
    return df_index
  

  def scatter_column_values(self, df, x_value, y_value):
    '''
    Function: Scatter plot between two columns in the same dataframe
    Input: Dataframe, x-value, y-value
    Output: Scatter plot
    Dependency: matplotlib.pyplot imported as plt
    '''
    begin = time.time()
    df.plot(x=x_value, y=[y_value], kind='scatter')
    print(x_value,'vs.',y_value)
    print('\n')
    plt.figure(figsize=(3, 4))
    plt.show()

    end = time.time()
      
    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)


    def print_outlier_values(self, df, column_name):
      '''
      Function: Find the higher and lower outlier values of a column.
                Get two dataframes with only higher and lower values rows
      Input: Dataframe and a column name
      Outout: Two dataframes with higher and lower outliers respectively
      '''
      begin = time.time()
      quartiles = twitter_archive[column_name].quantile([0.25,0.5,0.75])
      q1 = quartiles[0.25]
      q3 = quartiles[0.75]
      iqr = q3 - q1
      lower_outlier_filter = q1 - 1.5 * iqr
      higher_outlier_filter = q3 + 1.5 * iqr
      print('Higher outliers:',higher_outlier_filter)
      print('Lower outliers:',lower_outlier_filter)
      end = time.time()
        
      # Display the time taken to run this procedure
      self.find_time_taken(begin, end)


    def get_outlier_dfs(self, df, column_name):
      '''
      Function: Find the higher and lower outlier values of a column.
                Get two dataframes with only higher and lower values rows
      Input: Dataframe and a column name
      Outout: Two dataframes with higher and lower outliers respectively
      '''
      begin = time.time()
      quartiles = df[column_name].quantile([0.25,0.5,0.75])
      q1 = quartiles[0.25]
      q3 = quartiles[0.75]
      iqr = q3 - q1
      lower_outlier_filter = q1 - 1.5 * iqr
      higher_outlier_filter = q3 + 1.5 * iqr
      df_higher_outliers = df.loc[(df[column_name] > higher_outlier_filter)]
      df_lower_outliers = df.loc[(df[column_name] < lower_outlier_filter)]
      end = time.time()
        
      # Display the time taken to run this procedure
      self.find_time_taken(begin, end)
      return df_higher_outliers, df_lower_outliers
  

    def get_outlier_normal_dist(self, df, column_name):
      '''
      Function: Get higher outlier dataframe based on the given column that is distributed normally
      Input: Dataframe, column name
      Output: Higher outlier dataframe
      Dependency: from scipy.stats import zscore
      '''
      begin = time.time()
      df_copy = df.copy(deep=False)
      new_col = column_name + '_zscore'
      df_copy[new_col] = zscore(df[column_name])
      df_higher_outliers = df_copy.loc[(abs(df_copy[new_col]) > 3)]
      end = time.time()
        
      # Display the time taken to run this procedure
      self.find_time_taken(begin, end)
      return df_higher_outliers
    

  def check_decimal_places(self, df, column_list):
    '''
    Function: Check the number of decimal places in a given column
    Input: The dataframe, column_list
    Output: The unique number of decimal places found in the columns
    '''
    begin = time.time()
    for i in column_list:
      places = -np.floor(np.log10(df[i]))
      n_unique_places = places.unique()
      print('Column Name:',i)
      print(n_unique_places)
      print('\n')
    end = time.time()
    
    # Display the time taken to run this procedure
      
    self.find_time_taken(begin, end)


  def barchart(self, df, column_name, chart_type):
    '''
    Function: Create bar chart
    Input: The dataframe and column name
    Output: Bar chart showing the frequency
    Dependency: import seaborn as sb
    '''

    begin = time.time()
    if chart_type == 'abs':
      sb.set_theme(style="whitegrid")
      sb.set_color_codes("pastel")
      #pal = ['black', 'grey', 'grey', 'grey']
      pal = "ch:.25"
      sb.set_style(style='white')
      title_value = column_name + ' absolute frequency'
      ax = sb.countplot(data = df, x = column_name, palette=pal).set(title=title_value)
      #order = titanic['class'].value_counts().index

    else:
      title_value = column_name + ' relative frequency'
      proportions_column = round(df[column_name].value_counts()/len(df),3)
      df_1 = pd.DataFrame({'Unique Values':proportions_column.index, 'Percentage':proportions_column.values})
      ax = df_1.plot.bar(x='Unique Values', y='Percentage', title = title_value, legend=False)
      ax.set_xlabel(column_name)
      ax.set_ylabel("Relative Frequency")
      # Hide grids
      ax.grid(False)
      # Hide axes ticks
      ax.set_xticks([])
      ax.set_yticks([])
      #ax.plot(legend=False)
      print('\n')
      # Show proportions
      print('Proportion Values:')
      print(df_1)
      print('\n')
    end = time.time()
    
    # Display the time taken to run this procedure
      
    self.find_time_taken(begin, end)
  

  def get_column_lists(self, df):
    '''
    Function: Get the categorical and numerical column names of a dataframe as two lists
    Input: A dataframe
    Output: The categorical and numerical column name lists
    '''
    begin = time.time()

    numeric_columns = df.select_dtypes([np.number]).columns.tolist()
    columns = list(df)
    categorical_columns = []
    for i in columns:
      if i not in numeric_columns:
        categorical_columns.append(i)
      else:
        None
    end = time.time()
    
    # Display the time taken to run this procedure
      
    self.find_time_taken(begin, end)
    return numeric_columns, categorical_columns
  

  def unique_values_categorical(self, df, column_list):
    '''
    Function: Find the number of unique values in the column lists
    Input: The dataframe and column list
    Output: Column name and the number of unique values
    '''
    begin = time.time()
    for i in column_list:
      print('Column Name:',i)
      unique = df[i].unique()
      print(len(unique))
    end = time.time()
    
    # Display the time taken to run this procedure
      
    self.find_time_taken(begin, end)


  def bin_num_range_selector(self, df, column_name):
    '''
    Function: Find the minimum and maximum values of the column
    Input: Dataframe, column name
    Output: Maximum and minimum values in the dataframe
    '''
    begin = time.time()
    # Find the maximum value in the column
    maximum = df[column_name].max()
    # Find the minimum value in the column
    minimum = df[column_name].min()
    
    print('Maximum:',maximum)
    print('Minimum:',minimum)

    end = time.time()
    
    # Display the time taken to run this procedure
      
    self.find_time_taken(begin, end)
  

  def bin_num_helper(self, upper, lower):
    '''
    Function: Get bin size suggestions as a list
    Input: Upper and lower values. 
            Upper - Slightly higher than the maximum value of the column (51 if max is 50). Must be a whole number
            Lower - Slightly lower than the minimum value of the column (1 if min is 2). Must be a whole number
    Output: A list of bin size suggestions
    '''
    begin = time.time()
    range = upper - lower
    num_range = np.arange(1,20).tolist()

    # Create a list of all values where range is divisible by numbers from 1 to 20
    divisible_num = []
    for i in num_range:
      if range%i == 0:
        divisible_num.append(i)
    print('Bin size suggestions:',divisible_num)
    print('\nOr any bin size from the range 1 to 20')
    end = time.time()
    
    # Display the time taken to run this procedure
      
    self.find_time_taken(begin, end)


  def make_histogram(self, df, column_name, n_bin):
    '''
    Function: Display histogram
    Input: Dataframe, column name of a numeric column and number of bins
    Output: Histogram
    '''
    begin = time.time()
    n_bin = n_bin
    fig = plt.figure(figsize=(15,4))
    ax = plt.gca()
    counts, _, patches = ax.hist(df[column_name], bins=n_bin)
    for count, patch in zip(counts,patches):
        ax.annotate(str(int(count)), xy=(patch.get_x(), patch.get_height()))
    plt.xlabel(column_name+' bins') 
    plt.ylabel('Frequency') 
    plt.title(column_name)
    plt.show()
    end = time.time()
    
    # Display the time taken to run this procedure
      
    self.find_time_taken(begin, end)

  
  def find_substring_item(self, column_vals_list, substring_list):
    '''
    Function: Find the substring in the unique values of the column
    Input: Column values as a list
    Output: Elements of the list that contain the substring
    '''
    begin = time.time()
    for i in substring_list:
      print('\nItems with the substring:',i)
      #print('\n')
      for j in column_vals_list:
        if(i in j):
          print(j)

    end = time.time()
    
    # Display the time taken to run this procedure
      
    self.find_time_taken(begin, end)


  # Check the frequency of the column
  def frequency_col(self, df, column_name):
    '''
    Function: Create a dataframe with frequency of a column in the given dataframe
    Input: The dataframe, column name
    Output: The dataframe with frequencies of the given column name
    '''
    begin = time.time()
    frequency = df['CNT'].value_counts()
    end = time.time()
    
    # Display the time taken to run this procedure
      
    self.find_time_taken(begin, end)
    return frequency
  

  def frequency_table_groupby(self, df, column_name):
    '''
    Function: Group the table frequency with a column
    Input: The dataframe, column name
    Output: The dataframe with frequencies of all the columns grouped by a single column
    '''
    begin = time.time()
    frequency_df = df.groupby(column_name).count()
    end = time.time()
    self.find_time_taken(begin, end)
    return df


B - Assess 3 tables

In [7]:
# Class that helps the assessment of the three tables

class ThreeTableAssessment:
  
  def __init__(self, df1, df2, df3):
    self.df1 = df1
    self.df2 = df2
    self.df3 = df3


  def find_missing_rows(self, df1, df2, df3,id_1, id_2, id_3):
    
    # Find the number of ids which are missing from in second and third dataframes compared to the first
    # It is assumed the first dataframe has all the required data points
    begin = time.time()

    ta_id = df1[id_1].to_list()
    im_id = df2[id_2].to_list()
    td_id = df3[id_3].to_list()

    ta_im = list(set(im_id) ^ set(ta_id))
    ta_td = list(set(td_id) ^ set(ta_id))
    im_td = list(set(td_id) ^ set(im_id))

    print('The number of uncommon rows between first and second dataframes:',len(ta_im))
    print('The number of uncommon rows between first and third dataframes:',len(ta_td))
    print('The number of uncommon rows between second and third dataframes:',len(im_td))
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)
    


  # Function to calculate the time taken to run the function that calls it
  def find_time_taken(self, begin, end):
    '''
    Intention: Calculate the time taken to run this procedure
    Input: Beginning time, ending time
    Action: Calculate the time taken for a procedure to be run in minutes or seconds
    '''
    time_taken = end - begin

    if time_taken >= 60:
      time_taken = round(time_taken/60,2)
      print("Time taken to run this procedure:",time_taken,"minutes")
    else:
      time_taken = round(time_taken,2)
    print('\n')
    print("Time taken to run this procedure:",time_taken,"seconds")
  

  # Check for the same column headers in multiple columns
  def find_duplicate_headers(self, df1, df2, df3):
    '''
    Function - To find the common column names within three dataframes
    Input: Three dataframes
    Output: Common columns within each dataframes 
    '''

    begin = time.time()
    # Between df1 and df2
    c1 = df1[df1.columns.intersection(df2.columns)]

    # Between df2 and df3
    c2 = df2[df2.columns.intersection(df3.columns)]

    # Between df3 and df1
    c3 = df3[df3.columns.intersection(df1.columns)]

    print('The common columns between the first two data frames are:')
    print(c1.columns)
    print('\nThe common columns between the second and third data frames are:')
    print(c2.columns)
    print('\nThe common columns between the third and first data frames are:')
    print(c3.columns)
    end = time.time()

    # Display the time taken to run this procedure
    self.find_time_taken(begin, end)





2 - Create an instance of the class

In [8]:
# Instance of DataAssessment class
instance = DataAssessment(df_sales_train)

# Instance of ThreeTableAssessment class
instance_1 = ThreeTableAssessment(df_sales_train, df_item_categories, df_items)

## 2. Asess Data

1.Check basic file information

In [ ]:
instance.files_basic_info()

The number of files in the directory is: 8


The file names are:
['items.csv', 'item_categories.csv', 'sales_train.csv', 'test.csv', 'shops.csv', 'sample_submission.csv', 'kaggle.json', 'competitive-data-science-predict-future-sales.zip']


The csv files details are:


File: items.csv
File size: 1.50 MB
Number of data points: 22170
Number of features: 3


File: item_categories.csv
File size: 0.00 MB
Number of data points: 84
Number of features: 2


File: sales_train.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  """Entry point for launching an IPython kernel.


File size: 90.22 MB
Number of data points: 2935849
Number of features: 6


File: test.csv
File size: 3.04 MB
Number of data points: 214200
Number of features: 3


File: shops.csv
File size: 0.00 MB
Number of data points: 60
Number of features: 2


File: sample_submission.csv
File size: 2.14 MB
Number of data points: 214200
Number of features: 2




Time taken to run this procedure: 2.54 seconds


2. Print dataframe details horizontally

1. The df_sales_train dataframe

In [ ]:
instance.print_head(df_sales_train)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


In [ ]:
instance.print_vertically(df_sales_train)

date              02.01.2013
date_block_num             0
shop_id                   59
item_id                22154
item_price             999.0
item_cnt_day             1.0
Name: 0, dtype: object

2. The df_shops dataframe

In [ ]:
instance.print_head(df_shops)

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


3. The df_items

In [ ]:
instance.print_head(df_items)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
5,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,5,40
6,***УДАР ПО ВОРОТАМ (UNI) D,6,40
7,***УДАР ПО ВОРОТАМ-2 (UNI) D,7,40
8,***ЧАЙ С МУССОЛИНИ D,8,40
9,***ШУГАРЛЭНДСКИЙ ЭКСПРЕСС (UNI) D,9,40


4. The df_item_categories

In [ ]:
instance.print_head(df_item_categories)

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
5,Аксессуары - PSVita,5
6,Аксессуары - XBOX 360,6
7,Аксессуары - XBOX ONE,7
8,Билеты (Цифра),8
9,Доставка товара,9


5. The df_sample_submission

In [ ]:
instance.print_head(df_sample_submission) 

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
5,5,0.5
6,6,0.5
7,7,0.5
8,8,0.5
9,9,0.5


In [9]:
instance.print_head(df_test) 

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
5,5,5,5039
6,6,5,5041
7,7,5,5046
8,8,5,5319
9,9,5,5003


## 3. Conlclusions

#### 1. Insights
1. There are 4 data files given
2. The sales_train.csv file seems to be the main one with shops.csv, items.csv and item_categories.csv complementing it
3. The df_sales_train 
  1.  It is the training set. Daily historical data from January 2013 to October 2015
  2. Each observation shows the number of sale of an item in a shop
  2. Has two million nine hundred thirty-five thousand eight hundred forty-nine observations in 6 features
  3. There are 2 ids - shop_id and item_id
  4. There is a date column
  5. The date_block_num shows a block of days January 2013 - 0, February 2013 - 1 etc. It is a consecutive month number, used for convenience
  6. **item_cnt_day** - number of products sold. You are predicting a monthly amount of this measure
  7. The item_price feature is the price of a single item and not the cummulative sales value
4. df_shops
  1. There are 60 observations and 2 features
  2. This dataframe has the name of the shops and their ids
5. df_items
  1. There are 22,170 observations and 2 features
  2. This dataframe has the name of the items and their ids
6. df_item_categories
  1. There are 84 observations and 2 features
  2. This dataframe has the name of the item_categories and their ids




#### 2. Action Items

**Basic Cleaning**

1. To analyze data with only id numbers for shops, items and item categories, use df_sales_train
  1. Using the date_block_num combine the per day sales of items to per month sales of each shop
2. To combine textual information, combine shop_id from df_shops, item_id from df_items and item_category_id from df_item_catories to create df_2

**df_sales_train**

1. Split *date* column to day, month and year
2. Look at the item_cnt_day closesly. These are returns. Analyse how that works and the returned products closesly
3. Check if floating points of price column is rounded to two
4. Check the number of days in each date_block_num
5. Verify between if date_num_block is marked properly
6. Add the feature *item_total_sales* which is item_price *item_cnt_day*
7. Add classes for *item_price* and *item_total_sales*

**df_shops**

1. Check if all the shops mentioned in this dataframe appears in df_sales_train dataframe
2. Split the shop_name feature according to punctuation into multiple features

**df_items**

1. Check if all the items and item categories in this file appear in the df_sales train
2. Check if all the items and item categories are mapped to the same way in both this file and in df_sales_train
3. Split item_name into multiple features depending on the punctuation

**df_item_categories**

1. Check if the item categories in this file appear in the df_sales train
2. Check if all the items and item categories are mapped to the same way in both this file, in df_sales_train and in df_items
3. Split item_name into multiple features depending on the punctuation

#### 3. Task

The final submission file contains an ID feature (the shop id) and the feature item_cnt_month, which is the total items sold in a month